In [73]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

In [106]:
device= torch.device("cuda"if torch.cuda.is_available else "cpu")
image_size = 365

epoches= 10000
learning_rate= 0.001
alpha= 1
beta=0.01
device

device(type='cuda')

In [107]:
class VGG(nn.Module):
  def __init__(self) :
    super(VGG,self).__init__()
    #we are chossing the first conv layer after the max pooling
    self.chosen_features = ['0','5','10','19','28']
    #we are slelcting only upto 28 layers(29-1) for our model
    self.model= models.vgg19(pretrained=True).features[:29]

  def forward(self,x):
    #getting chosen feature layers from model
    features=[]
    for layer_num,layer in enumerate(self.model):
      x=layer(x)
      if str(layer_num) in self.chosen_features:
        features.append(x)
    return features


In [108]:
#we are loading unsquzeeing the image and moving it to device ie. GPU or CPU
def load_image(image_name):
  image=Image.open(image_name)
  image=loader(image).unsqueeze(0)
  return image.to (device)

In [109]:
loader=transforms.Compose(
    [
        transforms.Resize((image_size,image_size)),
        transforms.ToTensor()
    ]
)

In [125]:
original_image=load_image("mona_lisa.PNG")
style_image=load_image("stary_nights.JPG")

In [126]:
model=VGG().to(device).eval()
generated=original_image.clone().requires_grad_(True)
generated.shape

torch.Size([1, 3, 365, 365])

In [127]:
#setting optimizer
optimizer=optim.Adam([generated],lr=learning_rate)

In [128]:
#training loop
for steps in range(epoches):
  generated_features=model(generated)
  original_features=model(original_image)
  style_features=model(style_image)

  style_loss = original_loss = 0

  for gen_features,orgin_features,sty_features in zip(generated_features,original_features,style_features):
    batch_size,channel,height,width=gen_features.shape
    original_loss += torch.mean((gen_features-orgin_features)**2)

    #compute gram matrix
    G = gen_features.view(channel,height*width).mm(gen_features.view(channel,height*width).t())
    
    A = sty_features.view(channel,height*width).mm(sty_features.view(channel,height*width).t())

    style_loss += torch.mean((G-A)**2)


  total_loss = alpha * original_loss + beta * style_loss
  optimizer.zero_grad()
  total_loss.backward()
  optimizer.step()

  if steps % 100 == 0:
    print(steps,total_loss)
    save_image(generated,"genarated.png")

0 tensor(784119.1250, device='cuda:0', grad_fn=<AddBackward0>)
100 tensor(55781.5664, device='cuda:0', grad_fn=<AddBackward0>)
200 tensor(33027.6328, device='cuda:0', grad_fn=<AddBackward0>)
300 tensor(23847.5566, device='cuda:0', grad_fn=<AddBackward0>)
400 tensor(17558.4629, device='cuda:0', grad_fn=<AddBackward0>)
500 tensor(12659.5508, device='cuda:0', grad_fn=<AddBackward0>)
600 tensor(8840.7246, device='cuda:0', grad_fn=<AddBackward0>)
700 tensor(6102.9321, device='cuda:0', grad_fn=<AddBackward0>)
800 tensor(4351.3506, device='cuda:0', grad_fn=<AddBackward0>)
900 tensor(3338.3030, device='cuda:0', grad_fn=<AddBackward0>)
1000 tensor(2775.9060, device='cuda:0', grad_fn=<AddBackward0>)
1100 tensor(2446.6453, device='cuda:0', grad_fn=<AddBackward0>)
1200 tensor(2230.3333, device='cuda:0', grad_fn=<AddBackward0>)
1300 tensor(2069.2034, device='cuda:0', grad_fn=<AddBackward0>)
1400 tensor(1938.3173, device='cuda:0', grad_fn=<AddBackward0>)
1500 tensor(1828.8344, device='cuda:0', grad_